# Generative Adversarial Networks

Generative Andverarsarial Networks, [introduced](https://arxiv.org/abs/1406.2661) by Ian Goodfellow in 2014 have been the most powerfull model for generating any types from data from images ot text.

This notebook will teach you the basics of generative adversarial models.

In [ ]:
import theano
import theano.tensor as T

import lasagne
from lasagne.layers import *

## 1) Define our Generator

GANs consist of two models a generator, which generates data samples and a discriminator, which distinguishes samples generated by the generator from the original samples from the dataset.

The generator learns the mapping between a latent space and a given data distripution. As an input it must recieve some kind of noise, it could be just a random distribution (in our tutorial we will use uniform noise) or it could be a condition (onehot vector, image or sentence fed through a dropout layer).  

In [ ]:
#you can play around with the hyperparams as you want

class generator:
    input_noise = T.matrix("Noise")
    input_shape = [None, 32]
    
    inp = InputLayer(input_shape, input_noise)
    hid = DenseLayer(inp, 128)
    out = <no nonlinearity, shape of your choice>
    
    fake_gaussian = get_output(out)
    
    weights = get_all_params(out, trainable=True)

## 1.2) Define our Discriminator

The discriminator distinguishes generated samples from real ones by giving the labels 0, meaning the given sample is fake, and 1 meaning the opposite.

In [ ]:
real_gaussian = T.matrix("Target gaussian")

In [ ]:
class discriminator:
    inp = InputLayer(generator.out.output_shape)
    hid = DenseLayer(inp, 64)
    prob = <probabilities of sample being real>
    
    prob_fake = get_output(prob, {inp : generator.fake_gaussian})
    prob_real = get_output(prob, {inp : real_gaussian})
    
    weights = get_all_params(prob)

## 2) Define our loss functions 

We train the generator to minimize $$-\log(D(G(z)))$$

And we train the discriminator to minimize $$-(\log(x) + \log(1 - G(z))$$

In [ ]:
d_loss = -(T.log(discriminator.prob_real) + T.log(1 - discriminator.prob_fake)).mean()
g_loss = -(T.log(discriminator.prob_fake)).mean()

d_updates = lasagne.updates.adam(d_loss, discriminator.weights)
g_updates = lasagne.updates.adam(g_loss, generator.weights)

d_train = theano.function([generator.input_noise, real_gaussian], d_loss, updates=d_updates,
                          allow_input_downcast=True)
g_train = theano.function([generator.input_noise], g_loss, updates=g_updates, allow_input_downcast=True)

sample = theano.function([generator.input_noise], generator.fake_gaussian, allow_input_downcast=True)

## A little helper function

In [ ]:
import numpy as np

def sample_batch(batch_size):
    x = np.random.uniform(-10, 10, size=[batch_size] + list(generator.input_shape[1:]))
    y = np.random.normal(size=[batch_size] + list(generator.out.output_shape[1:]))    
    
    return x, y

## 3) Training loop
    1) Train discriminator
    2) Train generator

In [ ]:
n_epochs = 2000
batch_size = 32

for ep in range(n_epochs):
    x, y = sample_batch(batch_size)
    
    d_train(x, y)
    
    g_train(x)

In [ ]:
import matplotlib.pyplot as plt

x, y = sample_batch(1)
plt.hist(sample(x)[0])